In [1]:
import pandas as pd
import numpy as np
import requests
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import statsmodels.api as sm
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from tpot import TPOTClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder
from tpot import TPOTClassifier
from sklearn.tree import DecisionTreeClassifier
from mord import OrdinalRidge
import xgboost as xgb
from keras.layers import Input
from keras_tuner import RandomSearch
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import shap
from tabulate import tabulate
import requests
from io import StringIO

In [2]:
# The URL of the raw dataset on GitHub
url = "https://raw.githubusercontent.com/abactat/BC-Project/main/data/processed/train_data.csv?token=GHSAT0AAAAAACC4ZCNKRKQBWCGNKARGMRZ2ZGRO5TA"

# Send an HTTP GET request to fetch the content of the raw dataset
response = requests.get(url)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    # Read the content as a pandas DataFrame and assign it to the 'train' variable
    train = pd.read_csv(StringIO(response.text))
    print("Dataset downloaded and loaded into 'train' successfully.")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")
    
# The URL of the raw dataset on GitHub
url = "https://raw.githubusercontent.com/abactat/BC-Project/main/data/processed/val_data.csv?token=GHSAT0AAAAAACC4ZCNL6CLEYWQ7I647KVRWZGRO57A"

# Send an HTTP GET request to fetch the content of the raw dataset
response = requests.get(url)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    # Read the content as a pandas DataFrame and assign it to the 'valid' variable
    valid = pd.read_csv(StringIO(response.text))
    print("Dataset downloaded and loaded into 'valid' successfully.")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")
    
# The URL of the raw dataset on GitHub
url = "https://raw.githubusercontent.com/abactat/BC-Project/main/data/processed/test_data.csv?token=GHSAT0AAAAAACC4ZCNKRR63P6ASKCCXTCAWZGRO6HQ"

# Send an HTTP GET request to fetch the content of the raw dataset
response = requests.get(url)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    # Read the content as a pandas DataFrame and assign it to the 'train' variable
    test = pd.read_csv(StringIO(response.text))
    print("Dataset downloaded and loaded into 'test' successfully.")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")

Dataset downloaded and loaded into 'train' successfully.
Dataset downloaded and loaded into 'valid' successfully.
Dataset downloaded and loaded into 'test' successfully.


In [3]:
# Convert variables to numeric in the train dataset
variables_to_convert = train.columns.drop('Date')
train[variables_to_convert] = train[variables_to_convert].apply(pd.to_numeric, errors='coerce')

# Convert variables to numeric in the valid dataset
valid[variables_to_convert] = valid[variables_to_convert].apply(pd.to_numeric, errors='coerce')

In [4]:
train['Direction'] = np.select([
    train['Difference'] > 0,
    train['Difference'] < 0,
    train['Difference'] == 0
], [
    1,  # Increase
    -1,  # Decrease
    0   # Hold
], default=-1)

valid['Direction'] = np.select([
    valid['Difference'] > 0,
    valid['Difference'] < 0,
    valid['Difference'] == 0
], [
    1,  # Increase
    -1,  # Decrease
    0   # Hold
], default=-1)

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


In [5]:
print(train.columns)

Index(['Increase', 'Decrease', 'Retail Sales', 'Durable Goods Orders', 'CEI',
       'LAG', 'Net Sentiment Score', 'Positive Frequency',
       'Negative Frequency', 'Proportion Negative Words',
       ...
       'unwilling', 'vulnerable', 'weak', 'weakening', 'weaker', 'weakest',
       'weakness', 'Difference', 'Date', 'Direction'],
      dtype='object', length=103)


###Directional Prediction

###Ordinal Ridge Regression

In [6]:
# Prepare the data for the model
X_train = train.drop(columns=['Difference', 'Increase', 'Decrease', 'Direction', 'Date'])
y_train = train['Direction']  # Use the 'Direction' variable as the target

X_valid = valid.drop(columns=['Difference', 'Increase', 'Decrease', 'Direction', 'Date'])
y_valid = valid['Direction']  # Use the 'Direction' variable as the target

# Center and scale the independent variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Initialize and train the ordinal logistic regression model
ordinal_model = OrdinalRidge()
ordinal_model.fit(X_train_scaled, y_train)

# Make predictions on the validation set
y_valid_pred = ordinal_model.predict(X_valid_scaled)

# Make predictions on the training set
y_train_pred = ordinal_model.predict(X_train_scaled)

In [7]:
# Calculate Mean Squared Error (MSE) for training set
mse_train = mean_squared_error(y_train, y_train_pred)
print("Training Mean Squared Error (MSE):", mse_train)

# Calculate Mean Squared Error (MSE) for validation set
mse_valid = mean_squared_error(y_valid, y_valid_pred)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training set
rmse_train = np.sqrt(mse_train)
print("Training Root Mean Squared Error (RMSE):", rmse_train)

# Calculate Root Mean Squared Error (RMSE) for validation set
rmse_valid = np.sqrt(mse_valid)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training set
r2_train = r2_score(y_train, y_train_pred)
print("Training R^2:", r2_train)

# Calculate R-squared (R^2) for validation set
r2_valid = r2_score(y_valid, y_valid_pred)
print("Validation R^2:", r2_valid)

# Calculate adjusted R-squared for training set
n_train = X_train.shape[0]  # Number of samples in training set
p_train = X_train.shape[1]  # Number of features in training set
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
print("Training Adjusted R^2:", adj_r2_train)

# Calculate adjusted R-squared for validation set
n_valid = X_valid.shape[0]  # Number of samples in validation set
p_valid = X_valid.shape[1]  # Number of features in validation set
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Validation Adjusted R^2:", adj_r2_valid)

Training Mean Squared Error (MSE): 0.171875
Validation Mean Squared Error (MSE): 0.4166666666666667
Training Root Mean Squared Error (RMSE): 0.414578098794425
Validation Root Mean Squared Error (RMSE): 0.6454972243679028
Training R^2: 0.41673570836785423
Validation R^2: -0.016949152542373058
Training Adjusted R^2: -0.19788687851333164
Validation Adjusted R^2: 1.311864406779661


In [8]:
# Initialize and train the ordinal logistic regression model
ordinal_model = OrdinalRidge()
ordinal_model.fit(X_train_scaled, y_train)

# Make predictions on the validation set
y_valid_pred = ordinal_model.predict(X_valid_scaled)

# Make predictions on the training set
y_train_pred = ordinal_model.predict(X_train_scaled)

# Convert the predicted values to ordinal categories
y_train_pred_category = np.round(y_train_pred).astype(int)
y_valid_pred_category = np.round(y_valid_pred).astype(int)

# Calculate accuracy for training and validation sets
accuracy_train = accuracy_score(y_train, y_train_pred_category)
accuracy_valid = accuracy_score(y_valid, y_valid_pred_category)

print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

Training Accuracy: 0.828125
Validation Accuracy: 0.5833333333333334


In [9]:
# Access the coefficients and intercepts
coefficients = ordinal_model.coef_
intercepts = ordinal_model.intercept_

# Display coefficients and intercepts
print("Coefficients:")
print(coefficients)

print("Intercepts:")
print(intercepts)

Coefficients:
[-5.18872977e-02 -7.47103878e-03 -1.27142861e-01  4.39720469e-02
  4.92686532e-02  1.28221175e-01  5.94983164e-02 -1.44070405e-01
 -2.02847773e-01  4.92686532e-02  3.32961536e-01 -4.12040425e-02
  2.92831017e-02  3.67541886e-02  9.07505668e-02 -1.93300819e-02
  5.39659895e-03 -9.73120263e-02  6.24179548e-02  1.36183446e-02
 -2.05951765e-01  0.00000000e+00 -1.78430873e-02  1.81735631e-03
 -2.31545229e-02 -3.12228595e-04  1.86912293e-01 -1.58644961e-01
  2.72107726e-01  5.12002733e-02 -1.59608013e-01 -4.46777014e-02
  1.04121362e-02 -4.02629182e-02  5.07058500e-03  0.00000000e+00
  1.07862133e-02 -1.10251781e-01 -5.18124772e-03 -1.05490602e-01
  9.16206841e-03 -5.36278154e-02  2.03374856e-01 -7.33697411e-02
  1.02270689e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7.88688154e-03  5.51241238e-02  2.11119111e-02 -1.16108984e-02
  1.04702364e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -1.82273345e-02 -1.01895146e-02  1.32270029e-01  3.90890241e-03
 -4.6106250

###XGBoost Model

In [10]:
# Prepare the data for the model
X_train = train.drop(columns=['Difference', 'Increase', 'Decrease', 'Direction', 'Date'])
y_train = train['Direction']  # Use the 'Direction' variable as the target

X_valid = valid.drop(columns=['Difference', 'Increase', 'Decrease', 'Direction', 'Date'])
y_valid = valid['Direction']  # Use the 'Direction' variable as the target

# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Make predictions on the training set
y_train_pred = xgb_model.predict(X_train)

# Make predictions on the validation set
y_valid_pred = xgb_model.predict(X_valid)

In [11]:
# Calculate Mean Squared Error (MSE) for training set
mse_train = mean_squared_error(y_train, y_train_pred)
print("Training Mean Squared Error (MSE):", mse_train)

# Calculate Mean Squared Error (MSE) for validation set
mse_valid = mean_squared_error(y_valid, y_valid_pred)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training set
rmse_train = np.sqrt(mse_train)
print("Training Root Mean Squared Error (RMSE):", rmse_train)

# Calculate Root Mean Squared Error (RMSE) for validation set
rmse_valid = np.sqrt(mse_valid)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training set
r2_train = r2_score(y_train, y_train_pred)
print("Training R^2:", r2_train)

# Calculate R-squared (R^2) for validation set
r2_valid = r2_score(y_valid, y_valid_pred)
print("Validation R^2:", r2_valid)

# Calculate adjusted R-squared for training set
n_train = X_train.shape[0]  # Number of samples in training set
p_train = X_train.shape[1]  # Number of features in training set
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
print("Training Adjusted R^2:", adj_r2_train)

# Calculate adjusted R-squared for validation set
n_valid = X_valid.shape[0]  # Number of samples in validation set
p_valid = X_valid.shape[1]  # Number of features in validation set
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Validation Adjusted R^2:", adj_r2_valid)

Training Mean Squared Error (MSE): 3.792103751738781e-07
Validation Mean Squared Error (MSE): 0.24622443928668325
Training Root Mean Squared Error (RMSE): 0.0006158005969255617
Validation Root Mean Squared Error (RMSE): 0.4962100757609455
Training R^2: 0.9999987131352969
Validation R^2: 0.39904543631724754
Training Adjusted R^2: 0.9999973570843194
Validation Adjusted R^2: 1.1842927328627106


In [12]:
# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Extract feature importance
feature_importance = xgb_model.feature_importances_

# Create a dictionary to map feature names to their importance scores
feature_importance_dict = {feature_name: importance_score for feature_name, importance_score in zip(X_train.columns, feature_importance)}

# Sort the feature importance dictionary in descending order based on importance scores
sorted_feature_importance = dict(sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True))

# Print the feature importance scores in descending order
print("Feature Importance:")
for feature_name, importance_score in sorted_feature_importance.items():
    print(f"{feature_name}: {importance_score:.4f}")

Feature Importance:
unsustainable: 0.1893
Short-Term Treasury Diff: 0.1786
delayed: 0.1649
dissented: 0.0557
easing: 0.0392
LAG: 0.0361
strain: 0.0326
decline: 0.0323
weak: 0.0320
weakening: 0.0290
LAG_RollingMean: 0.0287
Proportion Negative Words: 0.0218
Net Sentiment Score: 0.0213
Positive Frequency: 0.0162
weaker: 0.0157
rebound: 0.0139
dropped: 0.0136
CEI_RollingMean: 0.0124
encouragement: 0.0100
Word Count: 0.0083
weakest: 0.0073
satisfactory: 0.0068
adversely: 0.0060
cutback: 0.0049
downturn: 0.0032
Retail Sales: 0.0029
attractive: 0.0027
CEI: 0.0025
deterioration: 0.0022
LEI_RollingMean: 0.0020
depress: 0.0015
Durable Goods Orders: 0.0013
sluggish: 0.0009
sharply: 0.0007
recession: 0.0006
stabilization: 0.0006
progressed: 0.0005
opportunity: 0.0003
downward: 0.0003
concern: 0.0003
adverse: 0.0003
Negative Frequency: 0.0002
persist: 0.0001
stronger: 0.0001
correction: 0.0001
contraction: 0.0001
Durable Goods Orders_RollingMean: 0.0001
improved: 0.0001
depressing: 0.0000
diminishe

In [13]:
# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Make predictions on the training set
y_train_pred = xgb_model.predict(X_train)

# Make predictions on the validation set
y_valid_pred = xgb_model.predict(X_valid)

# Convert the predicted values to ordinal categories
y_train_pred_category = np.round(y_train_pred).astype(int)
y_valid_pred_category = np.round(y_valid_pred).astype(int)

# Calculate accuracy for training and validation sets
accuracy_train = accuracy_score(y_train, y_train_pred_category)
accuracy_valid = accuracy_score(y_valid, y_valid_pred_category)

print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

Training Accuracy: 1.0
Validation Accuracy: 0.625


###RNN Model

In [14]:
# Preprocess the data
X_train = train.drop(columns=['Direction', 'Increase', 'Decrease', 'Difference', 'Date'])
y_train = train['Direction']

X_valid = valid.drop(columns=['Direction', 'Increase', 'Decrease', 'Difference', 'Date'])
y_valid = valid['Direction']

# Encode the ordinal 'Direction' variable to numerical values (e.g., -1, 0, 1)
y_train_encoded = pd.Series(np.where(y_train == 'Decrease', -1, np.where(y_train == 'Hold', 0, 1)))
y_valid_encoded = pd.Series(np.where(y_valid == 'Decrease', -1, np.where(y_valid == 'Hold', 0, 1)))

# Center and scale the independent variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Build the RNN model with additional modifications to obtain feature importances
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units', min_value=32, max_value=128, step=32),
                   input_shape=(sequence_length, 1),
                   return_sequences=True))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=16, max_value=64, step=16)))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4])),
                  metrics=['accuracy'])

    return model

# Prepare the input sequences for the RNN model (assuming sequence_length is defined)
def prepare_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length + 1):
        sequences.append(data[i:i+sequence_length])
        targets.append(data[i+sequence_length-1])
    return np.array(sequences), np.array(targets)

sequence_length = 10  # Define the length of the input sequence
X_train_seq, y_train_seq = prepare_sequences(y_train_encoded, sequence_length)
X_valid_seq, y_valid_seq = prepare_sequences(y_valid_encoded, sequence_length)

# Build the RNN model with additional modifications to obtain feature importances
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 1), return_sequences=True))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_valid_seq, y_valid_seq))

# Evaluate the model on the validation set
y_train_pred = model.predict(X_train_seq).argmax(axis=1)
y_valid_pred = model.predict(X_valid_seq).argmax(axis=1)

# Calculate Mean Squared Error (MSE) for training and validation sets
mse_train = mean_squared_error(y_train_seq, y_train_pred)
mse_valid = mean_squared_error(y_valid_seq, y_valid_pred)
print("Training Mean Squared Error (MSE):", mse_train)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training and validation sets
rmse_train = np.sqrt(mse_train)
rmse_valid = np.sqrt(mse_valid)
print("Training Root Mean Squared Error (RMSE):", rmse_train)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training and validation sets
r2_train = r2_score(y_train_seq, y_train_pred)
r2_valid = r2_score(y_valid_seq, y_valid_pred)
print("Training R^2:", r2_train)
print("Validation R^2:", r2_valid)

# Calculate the number of samples and features
n_train, p_train = X_train_seq.shape[0], X_train_seq.shape[1]
n_valid, p_valid = X_valid_seq.shape[0], X_valid_seq.shape[1]

# Calculate adjusted R-squared for training and validation sets
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Training Adjusted R^2:", adj_r2_train)
print("Validation Adjusted R^2:", adj_r2_valid)

# Show training and validation set accuracy
accuracy_train = (y_train_seq == y_train_pred).mean()
accuracy_valid = (y_valid_seq == y_valid_pred).mean()
print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

Epoch 1/10
6/6 [==============================] - 6s 238ms/step - loss: 0.8384 - accuracy: 1.0000 - val_loss: 0.6144 - val_accuracy: 1.0000
Epoch 2/10
6/6 [==============================] - 0s 20ms/step - loss: 0.4680 - accuracy: 1.0000 - val_loss: 0.2581 - val_accuracy: 1.0000
Epoch 3/10
6/6 [==============================] - 0s 20ms/step - loss: 0.1615 - accuracy: 1.0000 - val_loss: 0.0582 - val_accuracy: 1.0000
Epoch 4/10
6/6 [==============================] - 0s 22ms/step - loss: 0.0331 - accuracy: 1.0000 - val_loss: 0.0113 - val_accuracy: 1.0000
Epoch 5/10
6/6 [==============================] - 0s 21ms/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 6/10
6/6 [==============================] - 0s 23ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 7/10
6/6 [==============================] - 0s 20ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 9.4977e-04 - val_accuracy: 1.0000
Epoch 8/10
6/6 [======

In [15]:
# Build the RNN model with additional modifications to obtain feature importances
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units', min_value=32, max_value=128, step=32),
                   input_shape=(sequence_length, 1),
                   return_sequences=True))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=16, max_value=64, step=16)))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4])),
                  metrics=['accuracy'])

    return model

# Define the hyperparameter search space
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of hyperparameter combinations to try
    executions_per_trial=1,
    directory='my_dir',  # Directory to store the results
    project_name='my_project'
)

# Search for the best hyperparameters
tuner.search(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_valid_seq, y_valid_seq))

# Get the best model
best_model = tuner.get_best_models(1)[0]

# Print the summary of the best model
best_model.summary()

# Train the best model
best_model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_valid_seq, y_valid_seq))

# Evaluate the best model on the validation set
y_train_pred = best_model.predict(X_train_seq).argmax(axis=1)
y_valid_pred = best_model.predict(X_valid_seq).argmax(axis=1)

# Calculate Mean Squared Error (MSE) for training and validation sets
mse_train = mean_squared_error(y_train_seq, y_train_pred)
mse_valid = mean_squared_error(y_valid_seq, y_valid_pred)
print("Training Mean Squared Error (MSE):", mse_train)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training and validation sets
rmse_train = np.sqrt(mse_train)
rmse_valid = np.sqrt(mse_valid)
print("Training Root Mean Squared Error (RMSE):", rmse_train)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training and validation sets
r2_train = r2_score(y_train_seq, y_train_pred)
r2_valid = r2_score(y_valid_seq, y_valid_pred)
print("Training R^2:", r2_train)
print("Validation R^2:", r2_valid)

# Calculate the number of samples and features
n_train, p_train = X_train_seq.shape[0], X_train_seq.shape[1]
n_valid, p_valid = X_valid_seq.shape[0], X_valid_seq.shape[1]

# Calculate adjusted R-squared for training and validation sets
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Training Adjusted R^2:", adj_r2_train)
print("Validation Adjusted R^2:", adj_r2_valid)

# Show training and validation set accuracy
accuracy_train = (y_train_seq == y_train_pred).mean()
accuracy_valid = (y_valid_seq == y_valid_pred).mean()
print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

# Get the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print("Best Hyperparameters:")
print(best_hyperparameters.values)

INFO:tensorflow:Reloading Tuner from my_dir\my_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 3)                 99        
                                                                 
Total params: 12,771
Trainable params: 12,771
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
6/6 [==============================] - 5s 215ms/step - loss: 1.0067 - accuracy: 1.0000 - val_loss: 0.9953 - val_accuracy: 1.0000
Epoch 2/10
6/6 [==============================] - 0s 24ms/step - loss: 

In [16]:
# Build the RNN model with additional modifications to obtain feature importances
model = Sequential()
model.add(Input(shape=(sequence_length, 1)))  # Add an input layer with the desired input shape
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Show the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 10, 64)            16896     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 29,411
Trainable params: 29,411
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Convert 'Date' column to datetime type
train['Date'] = pd.to_datetime(train['Date'])
valid['Date'] = pd.to_datetime(valid['Date'])

# Define a reference date (you can choose any date you prefer)
reference_date = pd.to_datetime('1990-01-01')

# Extract numerical representation of 'Date' column
train['Date'] = (train['Date'] - reference_date).dt.days
valid['Date'] = (valid['Date'] - reference_date).dt.days

# Preprocess the data
X_train = train.drop(columns=['Direction', 'Increase', 'Decrease', 'Difference'])
y_train = train['Direction']

X_valid = valid.drop(columns=['Direction', 'Increase', 'Decrease', 'Difference'])
y_valid = valid['Direction']

# Encode the ordinal 'Direction' variable to numerical values (e.g., -1, 0, 1)
y_train_encoded = pd.Series(np.where(y_train == 'Decrease', -1, np.where(y_train == 'Hold', 0, 1)))
y_valid_encoded = pd.Series(np.where(y_valid == 'Decrease', -1, np.where(y_valid == 'Hold', 0, 1)))

# Center and scale the independent variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Build the RNN model with additional modifications to obtain feature importances
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units', min_value=32, max_value=128, step=32),
                   input_shape=(sequence_length, 1),
                   return_sequences=True))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=16, max_value=64, step=16)))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4])),
                  metrics=['accuracy'])

    return model

# Prepare the input sequences for the RNN model (assuming sequence_length is defined)
def prepare_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length + 1):
        sequences.append(data[i:i+sequence_length])
        targets.append(data[i+sequence_length-1])
    return np.array(sequences), np.array(targets)

sequence_length = 10  # Define the length of the input sequence
X_train_seq, y_train_seq = prepare_sequences(y_train_encoded, sequence_length)
X_valid_seq, y_valid_seq = prepare_sequences(y_valid_encoded, sequence_length)

# Build the RNN model with additional modifications to obtain feature importances
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 1), return_sequences=True))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_valid_seq, y_valid_seq))

# Evaluate the model on the validation set
y_train_pred = model.predict(X_train_seq).argmax(axis=1)
y_valid_pred = model.predict(X_valid_seq).argmax(axis=1)

# Calculate Mean Squared Error (MSE) for training and validation sets
mse_train = mean_squared_error(y_train_seq, y_train_pred)
mse_valid = mean_squared_error(y_valid_seq, y_valid_pred)
print("Training Mean Squared Error (MSE):", mse_train)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training and validation sets
rmse_train = np.sqrt(mse_train)
rmse_valid = np.sqrt(mse_valid)
print("Training Root Mean Squared Error (RMSE):", rmse_train)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training and validation sets
r2_train = r2_score(y_train_seq, y_train_pred)
r2_valid = r2_score(y_valid_seq, y_valid_pred)
print("Training R^2:", r2_train)
print("Validation R^2:", r2_valid)

# Calculate the number of samples and features
n_train, p_train = X_train_seq.shape[0], X_train_seq.shape[1]
n_valid, p_valid = X_valid_seq.shape[0], X_valid_seq.shape[1]

# Calculate adjusted R-squared for training and validation sets
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Training Adjusted R^2:", adj_r2_train)
print("Validation Adjusted R^2:", adj_r2_valid)

# Show training and validation set accuracy
accuracy_train = (y_train_seq == y_train_pred).mean()
accuracy_valid = (y_valid_seq == y_valid_pred).mean()
print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

Epoch 1/10
6/6 [==============================] - 6s 231ms/step - loss: 0.9230 - accuracy: 1.0000 - val_loss: 0.7140 - val_accuracy: 1.0000
Epoch 2/10
6/6 [==============================] - 0s 20ms/step - loss: 0.5651 - accuracy: 1.0000 - val_loss: 0.3362 - val_accuracy: 1.0000
Epoch 3/10
6/6 [==============================] - 0s 19ms/step - loss: 0.2142 - accuracy: 1.0000 - val_loss: 0.0786 - val_accuracy: 1.0000
Epoch 4/10
6/6 [==============================] - 0s 22ms/step - loss: 0.0455 - accuracy: 1.0000 - val_loss: 0.0170 - val_accuracy: 1.0000
Epoch 5/10
6/6 [==============================] - 0s 20ms/step - loss: 0.0112 - accuracy: 1.0000 - val_loss: 0.0058 - val_accuracy: 1.0000
Epoch 6/10
6/6 [==============================] - 0s 21ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 7/10
6/6 [==============================] - 0s 21ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 8/10
6/6 [==========